In [1]:
%matplotlib inline
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import glob
import dlib
import math

In [2]:
data_path = '../data'
emotions = ['neutral', 'happy', 'sad', 'fear', 'angry']

In [3]:
def shape_to_np(shape, dtype="int"):
    # initialize the list of (x, y)-coordinates
    coords = np.zeros((68, 2), dtype=dtype)

    # loop over the 68 facial landmarks and convert them
    # to a 2-tuple of (x, y)-coordinates
    for i in range(0, 68):
        coords[i] = (shape.part(i).x, shape.part(i).y)

    # return the list of (x, y)-coordinates
    return coords

In [4]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('../models/shape_predictor_68_face_landmarks.dat')

In [5]:
landmarks = {}
for emotion in emotions:
    img_paths = glob.glob(data_path + '/' + emotion + '/*.png')
    row = 0
    for image in  img_paths:
        col = 0
        sample = cv2.imread(image)
        width = 400
        (h, w) = sample.shape[:2]
        new_size = (width, int(h * width/float(w)))
        sample = cv2.resize(sample, new_size)
        rects = detector(sample, 0)
        if len(rects) > 0:
            rect = rects[0]
            shape = predictor(sample, rect)
            shape = shape_to_np(shape)
            if row == 0:
                landmarks[emotion] = np.zeros((1, 68*68))
            else:
                landmarks[emotion] = np.vstack((landmarks[emotion], np.zeros((1, 68*68))))
            for i in range(shape.shape[0]):
                for j in range(shape.shape[0]):
                    dist =  math.sqrt((shape[j][0] - shape[i][0])**2 + (shape[j][1] - shape[i][1])**2)
                    landmarks[emotion][row, col] = dist
                    col += 1
            row += 1                

In [6]:
for i, emotion in enumerate(emotions):
    y = np.ones((landmarks[emotion].shape[0], 1))
    y *= i
    landmarks[emotion] = np.hstack((landmarks[emotion], y))
    np.save(data_path + '/emotion-landmarks/{}_landmarks'.format(emotion), landmarks[emotion])

In [7]:
data = np.copy(landmarks[emotions[0]])
for i in range(1, len(emotions)):
    data = np.vstack((data, landmarks[emotions[i]]))
np.save(data_path + '/data_set', data)